In [1]:
import sys
sys.path.append('..')

In [16]:
import numpy as np

import torch

import conversions as tgm

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [31]:
cons_p0 = torch.Tensor([0.0, 0.0, 0.017, 0.14, 0.11, 0])
all_poses_t = cons_p0.repeat(2,100,1) # [seq_length, N, 6]
all_poses_t[0] *= -1 # forward motion is -
all_inv_poses_t = -all_poses_t.clone()
print(all_poses_t.shape)
print(all_poses_t[0,0])
print(all_poses_t[1,0])
N = all_poses_t.shape[1] #len(all_poses) # number of sequences
k=3

torch.Size([2, 100, 6])
tensor([-0.0000, -0.0000, -0.0170, -0.1400, -0.1100, -0.0000])
tensor([0.0000, 0.0000, 0.0170, 0.1400, 0.1100, 0.0000])


In [32]:
ate_bs = []
ate_fs = []
f_pred_xyz = None
i=0
#for i in range(k):
idx = torch.arange(i, N, k)
print(idx)

tensor([ 0,  3,  6,  9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51,
        54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99])


In [34]:
b_pose_avg = (all_poses_t[0, idx] - all_inv_poses_t[0, idx])/2 # (tgt2src + inv(src2tgt))/2 [n,6]
print(b_pose_avg[0])

tensor([-0.0000, -0.0000, -0.0170, -0.1400, -0.1100, -0.0000])


In [39]:
b_pose = tgm.rtvec_to_pose(b_pose_avg) # src2tgt [n,4,4]
print(b_pose[0:2])

tensor([[[ 0.9999,  0.0170,  0.0000, -0.1400],
         [-0.0170,  0.9999,  0.0000, -0.1100],
         [ 0.0000,  0.0000,  1.0000, -0.0000],
         [ 0.0000,  0.0000,  0.0000,  1.0000]],

        [[ 0.9999,  0.0170,  0.0000, -0.1400],
         [-0.0170,  0.9999,  0.0000, -0.1100],
         [ 0.0000,  0.0000,  1.0000, -0.0000],
         [ 0.0000,  0.0000,  0.0000,  1.0000]]])


In [40]:
b_pose = tgm.inv_rigid_tform(b_pose) # tgt2src [n,4,4]
print(b_pose[0:2])

tensor([[[ 0.9999, -0.0170,  0.0000,  0.1381],
         [ 0.0170,  0.9999,  0.0000,  0.1124],
         [ 0.0000,  0.0000,  1.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  1.0000]],

        [[ 0.9999, -0.0170,  0.0000,  0.1381],
         [ 0.0170,  0.9999,  0.0000,  0.1124],
         [ 0.0000,  0.0000,  1.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  1.0000]]])


In [41]:
b_pose_abs = rel2abs_traj(b_pose)
print(b_pose_abs[0:3])

tensor([[[ 0.9999, -0.0170,  0.0000,  0.1381],
         [ 0.0170,  0.9999,  0.0000,  0.1124],
         [ 0.0000,  0.0000,  1.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  1.0000]],

        [[ 0.9994, -0.0340,  0.0000,  0.2743],
         [ 0.0340,  0.9994,  0.0000,  0.2271],
         [ 0.0000,  0.0000,  1.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  1.0000]],

        [[ 0.9987, -0.0510,  0.0000,  0.4085],
         [ 0.0510,  0.9987,  0.0000,  0.3441],
         [ 0.0000,  0.0000,  1.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  1.0000]]])


In [42]:
b_pose_abs = align_to_origin(b_pose_abs)
print(b_pose_abs[0:3])

tensor([[[ 1.0000e+00,  0.0000e+00,  0.0000e+00, -1.4901e-08],
         [ 0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],

        [[ 9.9986e-01, -1.6998e-02,  0.0000e+00,  1.3811e-01],
         [ 1.6998e-02,  9.9986e-01,  0.0000e+00,  1.1236e-01],
         [ 0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],

        [[ 9.9942e-01, -3.3991e-02,  0.0000e+00,  2.7429e-01],
         [ 3.3991e-02,  9.9942e-01,  0.0000e+00,  2.2706e-01],
         [ 0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]]])


In [43]:
print(b_pose_abs[0:5])

tensor([[[ 1.0000e+00,  0.0000e+00,  0.0000e+00, -1.4901e-08],
         [ 0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],

        [[ 9.9986e-01, -1.6998e-02,  0.0000e+00,  1.3811e-01],
         [ 1.6998e-02,  9.9986e-01,  0.0000e+00,  1.1236e-01],
         [ 0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],

        [[ 9.9942e-01, -3.3991e-02,  0.0000e+00,  2.7429e-01],
         [ 3.3991e-02,  9.9942e-01,  0.0000e+00,  2.2706e-01],
         [ 0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],

        [[ 9.9870e-01, -5.0975e-02,  0.0000e+00,  4.0850e-01],
         [ 5.0975e-02,  9.9870e-01,  0.0000e+00,  3.4405e-01],
         [ 0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00, 

In [44]:
b_pose = (all_poses_t[0, idx] - all_inv_poses_t[0, idx])/2 # (tgt2src + inv(src2tgt))/2 [n,6]
# b_pose = tgm.rtvec_to_pose(b_pose_avg) # src2tgt [n,4,4]
# b_pose = tgm.inv_rigid_tform(b_pose) # tgt2src [n,4,4]
b_pose = -b_pose # tgt2src [n,6]
# b_inv_pose = tgm.rtvec_to_pose(all_inv_poses_t[0, idx]) # inv(src2tgt) [n,4,4]            
# b_pose = rel2abs_traj(b_pose)
b_pose = b_pose.cumsum(dim=0) # [n,6]
print(b_pose[:3])

tensor([[0.0000, 0.0000, 0.0170, 0.1400, 0.1100, 0.0000],
        [0.0000, 0.0000, 0.0340, 0.2800, 0.2200, 0.0000],
        [0.0000, 0.0000, 0.0510, 0.4200, 0.3300, 0.0000]])


In [45]:
b_pose = b_pose-b_pose[0]

In [46]:
print(b_pose[:3])

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0170, 0.1400, 0.1100, 0.0000],
        [0.0000, 0.0000, 0.0340, 0.2800, 0.2200, 0.0000]])


In [ ]:

# Previous src
b_pose_avg = (all_poses_t[0, idx] - all_inv_poses_t[0, idx])/2 # (src2tgt + inv(tgt2src))/2 [n,4,4]
b_pose = tgm.rtvec_to_pose(b_pose_avg) # src2tgt [n,4,4]
b_pose = tgm.inv_rigid_tform(b_pose) # tgt2src [n,4,4]
# b_inv_pose = tgm.rtvec_to_pose(all_inv_poses_t[0, idx]) # inv(src2tgt) [n,4,4]
b_pose = rel2abs_traj(b_pose)
gt_idx = idx+k #torch.arange(k+i, N+k, k)
gt_seq_i = self.gt[gt_idx,:]
ate_b, _ = self.calculate_ate(b_pose, gt_seq_i)
ate_bs.append(ate_b)

# Next src
f_pose = tgm.rtvec_to_pose(all_poses_t[1, idx]) # tgt2src [n,4,4]
f_inv_pose = tgm.rtvec_to_pose(all_inv_poses_t[1, idx]) # inv(tgt2src) [n,4,4]
f_inv_pose = tgm.inv_rigid_tform(f_inv_pose) # inv(inv(tgt2src)) [n,4,4]
f_pose = (f_pose + f_inv_pose)/2 # (tgt2src + inv(inv(tgt2src)))/2 [n,4,4]
f_pose = rel2abs_traj(f_pose)
gt_idx = idx+2*k #torch.arange(2*k, N+2*k, k)
gt_seq_i = self.gt[gt_idx,:]
ate_f, f_pred_xyz = self.calculate_ate(f_pose, gt_seq_i)
ate_fs.append(ate_f)

ate_bs = torch.cat(ate_bs)
ate_fs = torch.cat(ate_fs)

In [28]:
def rel2abs_traj(rel_pose):
    """Convert a given relative pose sequences to absolute pose sequences.

    Args:
        rel_pose (torch.Tensor): Relative pose sequence in the form of homogenous transformation matrix. Shape: [N,4,4]

    Returns:
        torch.Tensor: Absolute pose sequence in the form of homogenous transformation matrix. Shape: [N,4,4]
    """

    abs_pose = rel_pose.clone()
    for i in range(1,abs_pose.shape[0]):
        abs_pose[i] = abs_pose[i] @ abs_pose[i-1]

    return abs_pose

In [38]:

def align_to_origin(pose):
    """Aligns a given trajectory to the origin.

    Args:
        pose (torch.Tensor): Absolute trajectory in the form of homogenous transformation matrix. Shape: [N,4,4]

    Returns:
        torch.Tensor: Origin-aligned absolute trajectory.
    """

    aligned_pose = pose.clone()
    inv_pose0 = tgm.inv_rigid_tform(pose[0:1])
    aligned_pose = torch.matmul(aligned_pose, inv_pose0)
    return aligned_pose